1) Predict emotion scores for all lines in the data
2) For each season
- Plot the distribution of all emotion labels in that season
3) For each emotion label
- Plot the relative frequency of each emotion across all seasons

In [ ]:
%pip install transformers

In [13]:
from transformers import pipeline

In [ ]:
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=True)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [19]:
classifier(pred_texts)

KeyboardInterrupt: 

# Try this:

In [ ]:
%pip install torch
%pip install pandas
%pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

2024-04-15 21:06:12.049844: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 21:06:13.124614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
#obs if this wont work, try restarting the kernel 

# load tokenizer and model, create trainer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

/home/ucloud/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.256, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
#specify your filename
file_name = "../../../../../cds-lang-data/GoT-scripts/Game_of_Thrones_Script.csv"  # note: you can right-click on your file and copy-paste the path to it here
text_column = "Sentence"  # select the column in your csv that contains the text to be classified

# read in csv
df_pred = pd.read_csv(file_name)
pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [18]:
pred_texts

["What do you expect? They're savages. One lot steals a goat from another lot and before you know it, they're ripping each other to pieces.",
 "I've never seen wildlings do a thing like this. I've never seen a thing like this, not ever in my life.",
 'How close did you get?',
 'Close as any man would.',
 'We should head back to the wall.',
 'Do the dead frighten you?',
 "Our orders were to track the wildlings. We tracked them. They won't trouble us no more.",
 "You don't think he'll ask us how they died? Get back on your horse.",
 'Whatever did it to them could do it to us. They even killed the children.',
 "It's a good thing we're not children. You want to run away south, run away. Of course, they will behead you as a deserter … If I don't catch you first. Get back on your horse. I won't say it again.",
 'Your dead men seem to have moved camp.',
 'They were here.',
 'See where they went.',
 'What is it?',
 "It's …",
 "Go on. Father's watching.",
 'And your mother.',
 'Fine work, as al

In [ ]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [ ]:
# Run predictions
predictions = trainer.predict(pred_dataset)

# New method

In [10]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load your CSV file
data = pd.read_csv('../../../../../cds-lang-data/GoT-scripts/Game_of_Thrones_Script.csv')

# Assuming your CSV has a column named 'text' containing the text data
texts = data['Sentence'].tolist()

# Tokenize texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)

# Decode predictions
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# Map predicted labels to emotions
emotion_map = {0: 'anger', 1: 'joy', 2: 'optimism', 3: 'sadness'}
predicted_emotions = [emotion_map[label] for label in predicted_labels]

# Add predicted emotions to the DataFrame
data['predicted_emotion'] = predicted_emotions

# Save the updated DataFrame
data.to_csv('output_data.csv', index=False)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

# from online

In [11]:
# Import pandas module
import pandas as pd
# Define the file name and path
file_name = "../../../../../cds-lang-data/GoT-scripts/Game_of_Thrones_Script.csv"
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_name)
# Print the first five rows of the DataFrame
print(df.head())

  Release Date    Season    Episode     Episode Title          Name  \
0   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce   
1   2011-04-17  Season 1  Episode 1  Winter is Coming          will   
2   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce   
3   2011-04-17  Season 1  Episode 1  Winter is Coming          will   
4   2011-04-17  Season 1  Episode 1  Winter is Coming         gared   

                                            Sentence  
0  What do you expect? They're savages. One lot s...  
1  I've never seen wildlings do a thing like this...  
2                             How close did you get?  
3                            Close as any man would.  
4                   We should head back to the wall.  


In [16]:
from transformers import DistilBertTokenizer

In [17]:
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=True)

tokenizer = DistilBertTokenizer.from_pretrained('j-hartmann/emotion-english-distilroberta-base')

/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

# Obs idea: make a shortened version of dataset before running on the whole thing